This notebook primarily deals with creation of preprocessing functions that contain code to preprocess vectors and rasters. Heavy notebook. Loads of functions and code. 

Go section by section. Read the description of the section for sure before you decide to run anything.

Necessary libraries might have to be manually installed

In [ ]:
# Import necessary packages
import os
import matplotlib.pyplot as plt
import numpy as np
import json
from glob import glob
import pandas as pd

# the below might not be installed already for you in your local installation
from osgeo import gdal
from geojson import Polygon, Feature, FeatureCollection, dump
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from haversine import haversine, Unit
from rasterio.plot import show
import geopandas as gpd
from rasterio import transform, features
from rasterio.plot import show

from area import area

gdal.UseExceptions()

### To Convert CRS

Use the below low level function or simply use **rio.warp.transform_bounds()**. This helps us convert the CRS of a raster to a specific CRS.

Completely independent function. Don't need to run anything before this step or after this step.

In [ ]:
# open TIFF file and define path to save new TIFF file
tif_path = '/Users/ishannangia/Desktop/TfW/tifs and labels/new_dryDeciduousSite.tif'
output_tif_path = '/Users/ishannangia/Desktop/TfW/tifs and labels/new_dryDeciduousSite.tif'

dataset = rio.open(tif_path)
data_to_transform = dataset

In [ ]:
# input destination CRS and display current CRS
# destination CRS
dst_crs = 'EPSG:4326'
# dst_crs = 'EPSG:32643'

# display current CRS
data_to_transform.crs

In [ ]:
# calculating transform along with other parameters for conversion
transform, width, height = calculate_default_transform(data_to_transform.crs, dst_crs, 
                                                       data_to_transform.width, data_to_transform.height, 
                                                       *data_to_transform.bounds)

new_kwargs = data_to_transform.meta.copy()
new_kwargs.update({
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height
})

# implementing the reprojection
with rio.open(output_tif_path, 'w', **new_kwargs) as dst:
    for i in range(1, data_to_transform.count + 1):
        reproject(
            source=rio.band(data_to_transform, i),
            destination=rio.band(dst, i),
            src_transform=data_to_transform.transform,
            src_crs=data_to_transform.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )

In [ ]:
# to check: open new data and check what values the transformed edge of image coordinates show
transformed_data = rio.open(output_tif_path)
transformed_data.transform * (transformed_data.width, transformed_data.height) # longitude, latitude format

### Finding the Extent of the Raster

Finding the size of the area covered by the raster, if the raster has "epsg 4326" as the CRS.

Completely independent function. Don't need to run anything before this step or after this step.

In [ ]:
def get_extent_in_meters_raster(ds):
    left, bottom, right, top = ds.bounds
    # (lat, long format)
    ul = top, left
    ur = top, right
    bl = bottom, left
    br = bottom, right

    ul_to_ur = haversine(ul, ur, unit=Unit.METERS)
    bl_to_br = haversine(bl, br, unit=Unit.METERS)
    ul_to_bl = haversine(ul, bl, unit=Unit.METERS)
    ur_to_br = haversine(ur, br, unit=Unit.METERS)
    return ul_to_ur, bl_to_br, ul_to_bl, ur_to_br

In [ ]:
# input TIFF path
tif_path_extent = './del_this_full.tif'

In [ ]:
# open the dataset and check CRS
extent_dataset = rio.open(tif_path_extent)
extent_dataset.crs.data["init"]

In [ ]:
# get the distances between corner coordinates
ul_to_ur, bl_to_br, ul_to_bl, ur_to_br = get_extent_in_meters_raster(extent_dataset)

In [ ]:
extent_dataset.close()

### For Splitting TIF Files

Here, we split TIFF files into smaller TIFF files with a predefined pixel height and width.

Completely independent function. Don't need to run anything before this step or after this step.

In [ ]:
# defining TIFF and paths
input_tif = '../../Desktop/TfW/tifs and labels/new_dryDeciduousSite.tif'
output_path = '../../Desktop/TfW/tifs and labels/dryDeciduousSite_tifs/'
main_file_suffix = 'dryDeciduousSite'

gdal_dataset = gdal.Open(input_tif)

In [ ]:
# defining destination smaller image (tile) size in pixels
tile_size_x = 640 * 3 # in pixels
tile_size_y = tile_size_x

band = gdal_dataset.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize

In [ ]:
# for creating all smaller tifs using one larger tif given above parameters
for i in range(0, xsize+tile_size_x, tile_size_x):
    for j in range(0, ysize+tile_size_y, tile_size_y):
        current_file_name = f"{str(main_file_suffix)}_x_{str(i)}_y_{str(j)}.tif"
        output_file_path = os.path.join(output_path, current_file_name)
        _ = gdal.Translate(output_file_path, gdal_dataset, srcWin=[i, j, tile_size_x, tile_size_y])
        
        #### todo: be a little careful of this code
        unique_mask_values = np.unique(_.GetRasterBand(4).ReadAsArray())
        if (len(unique_mask_values) == 1) and (unique_mask_values[0]==0):
            os.remove(output_file_path)
        _.Close() ### needed for viewing and actually working with the file

In [ ]:
#### code for creating a single mini TIF file

# center_x = 2000
# center_y = 2000
# tile_size_x = 640 * 3
# tile_size_y = tile_size_x

# band = gdal_dataset.GetRasterBand(1)
# xsize = band.XSize
# ysize = band.YSize

# _ = gdal.Translate(output_path, gdal_dataset, srcWin=[center_x, center_y, tile_size_x, tile_size_y])
# _.Close() ### needed for viewing and actually working with the file

### checking the output
# small_dataset = rio.open(output_path)
# plt.imshow(np.moveaxis(small_dataset.read([1,2,3]), 0, 2))

# # getting sizes
# ul_to_ur, bl_to_br, ul_to_bl, ur_to_br = get_extent_in_meters_raster(small_dataset)

# print(ul_to_ur, ul_to_bl)
# small_dataset.close()

## Converting LabelMe Annotations to Final Vector Files

This section allows you to create vector geojson files from LabelMe annotations that you either have created by manually labelling or created via the prediction outputs of some segmentation model.

There are three mini-sections here. All follow each other in the order that is given here and can't be run independently.

### 1. Converting Individual Labelme Labels to Individual Geojson Files

This code helps in converting labelme labels to geojsons/vectors that can be used to downstream when working with the satellite model.

Not a completely independent snippet/section. Post running this you should run the next snippet.

In [ ]:
# run this block if one has predefined sites where labelme annotations are kept in folders
# each site has a folder called sitename_tifs inside which jsons are kept

sites = ["burntSite", "restorationSite", "chromoStrobeSite", "plantationSite"]

for site in sites:
    main_address = f"/Users/Ishannangia/Desktop/TfW/tifs and labels/{site}_tifs"
    for geojson in glob(os.path.join(main_address, "*.geojson"):
        os.remove(geojson)
        
    all_jsons = glob(os.path.join(main_address, "*.json"))

    for json_file in all_jsons:
        name_of_file = json_file.split(".")[-2]
        name_of_tif = name_of_file + '.tif'

        with open(json_file, "r") as file:
            f = json.load(file)

        ds = rio.open(name_of_tif)

        all_polys = {}
        for polygon in f['shapes']:
            pts = []
            name = polygon['label']
            for point in polygon['points']:
                pt = ds.transform * (point[0], point[1])
                pts.append(pt)
            current_poly = Polygon([pts])
            try:
                all_polys[name].append(Feature(geometry=current_poly, properties= {"name": name}))
            except:
                all_polys[name] = [Feature(geometry=current_poly, properties= {"name": name})]

        for i, j in all_polys.items():
            feature_collection = FeatureCollection(j)
            with open(f'{name_of_file}_{i}.geojson', 'w') as f:
                dump(feature_collection, f)

In [ ]:
# run this block if you just have a single folder in which jsons are kept along with tiff files used in the labelme
# labelling procedure

main_address = f"/Users/Ishannangia/Desktop/three_dis_labelme_files"

for geojson in glob(os.path.join(main_address, "*.geojson")):
    os.remove(geojson)

all_jsons = glob(os.path.join(main_address, "*.json"))

for json_file in all_jsons:
    name_of_file = json_file.split(".")[-2]
    name_of_tif = name_of_file + '.tif'

    with open(json_file, "r") as file:
        f = json.load(file)

    ds = rio.open(name_of_tif)

    all_polys = {}
    for polygon in f['shapes']:
        pts = []
        name = polygon['label']
        for point in polygon['points']:
            pt = ds.transform * (point[0], point[1])
            pts.append(pt)
        current_poly = Polygon([pts])
        try:
            all_polys[name].append(Feature(geometry=current_poly, properties= {"name": name}))
        except:
            all_polys[name] = [Feature(geometry=current_poly, properties= {"name": name})]

    for i, j in all_polys.items():
        feature_collection = FeatureCollection(j)
        with open(f'{name_of_file}_{i}.geojson', 'w') as f:
            dump(feature_collection, f)

### 2. Combining all GeoJSONs Together

The above step produces an individual geojson/vector file for each labelme annotation file/image present in the given dir. This step combines all the individual geojsons together to create a single geojson for each degradation indicator that is present in the given folder.

Not a completely independent snippet/section. This should only be run on the outputs of the previous snippet. Post running this you should run the next snippet.

In [ ]:
# here label_dict should be changed to change mapping of degradation indicators from how it is labelled currently
# to what is needed in the output geojsons. Consider our 4 DI and 3 DI cases.

# run this block if one has predefined sites and the first code block from the above step has been run.


label_dict = {
    "Distorted Image": "Distorted Image",
    'Canopy Gap: Shaded': 'Canopy Gap: Shaded',
    'Canopy Gap: Vegetated': 'Canopy Gap: Vegetated',
    "Canopy Gap: Bare Land": 'Canopy Gap: Bare Land',
    'Plantation': 'Plantation',
    'Potential Invasive': 'Potential Invasive',
    'Potential Creeper': 'Potential Creeper',
    'Cane': 'Cane'
}

cat_dict = dict()

for site in sites:
    input_folder = f"/Users/Ishannangia/Desktop/TfW/tifs and labels/{site}_tifs/"
    all_geojsons = glob(os.path.join(input_folder, '*.geojson')) ## collect all geojsons from that folder
    geojson_categories = [file_name.split('.')[-2].split('/')[-1].split('_')[-1] for file_name in all_geojsons]
    unique_categories = set(geojson_categories)
    
    print('For site:', site)
    
    for category in unique_categories:
        try:
            converted_cat = label_dict[category]
        except:
            continue
        cat_features = []
        
        for idx, file_cat in enumerate(geojson_categories):
            if file_cat == category: 
                with open(all_geojsons[idx], "r") as file:
                    f = json.load(file)
                cat_features.extend(f["features"])
                
        feature_collection = FeatureCollection(cat_features)
        with open(f'{os.path.join(input_folder, converted_cat)}_{site}.geojson', 'w') as f:
            dump(feature_collection, f)
            
        print(category)
        print("Converted to:", converted_cat)
        print("Number of Polygons:", len(cat_features))
        
        try:
            cat_dict[converted_cat].extend(areas)
        except:
            cat_dict[converted_cat] = areas

        # get area stats
        areas = [area(feat["geometry"]) for feat in cat_features]
        print("\nAREA STATS (in sq. metres)")
        print('Min:', min(areas))
        print('Max:', max(areas))
        print('Mean:', np.mean(areas))
        print('Std. Deviation:', np.std(areas))
        for q in np.arange(10, 100, 20):
            print(f"{q}th percentile:", np.percentile(areas, q))
        plt.hist(areas, bins=20)
        plt.ylabel("Frequency")
        plt.xlabel("Size of Polygon")
        plt.title(f'Histogram for {converted_cat}')
        plt.show()
        print("---")

In [ ]:
# here label_dict should be changed to change mapping of degradation indicators from how it is labelled currently
# to what is needed in the output geojsons. Consider our 4 DI and 3 DI cases.

# run this block if one doesn't have predefined sites and only one single folder where geojsons are present
# the second code block from the above step should be run for prepping the files for this step

input_folder = f"/Users/Ishannangia/Desktop/three_dis_labelme_files"

label_dict = {
    "Distorted Image": "Distorted Image",
    'Canopy Gap: Shaded': 'Canopy Gap: Shaded',
    'Canopy Gap: Vegetated': 'Canopy Gap: Vegetated',
    "Canopy Gap: Bare Land": 'Canopy Gap: Bare Land',
    'Plantation': 'Plantation',
#     'Potential Invasive': 'Potential Invasive',
#     'Potential Creeper': 'Potential Creeper',
#     'Cane': 'Cane'
}

label_dict = {
    "Distorted Image": "Distorted Image",
    "Bare Land": 'Bare Land',
    "Canopy Gap": 'Canopy Gap',
    "Plantation": 'Plantation',
}

cat_dict = dict()

all_geojsons = glob(os.path.join(input_folder, '*.geojson')) ## collect all geojsons from that folder
geojson_categories = [file_name.split('.')[-2].split('/')[-1].split('_')[-1] for file_name in all_geojsons]
unique_categories = set(geojson_categories)


for category in unique_categories:
    try:
        converted_cat = label_dict[category]
    except:
        continue
    cat_features = []

    for idx, file_cat in enumerate(geojson_categories):
        if file_cat == category: 
            with open(all_geojsons[idx], "r") as file:
                f = json.load(file)
            cat_features.extend(f["features"])

    feature_collection = FeatureCollection(cat_features)
    with open(f'{os.path.join(input_folder, converted_cat)}.geojson', 'w') as f:
        dump(feature_collection, f)

#     print(category)
#     print("Converted to:", converted_cat)
#     print("Number of Polygons:", len(cat_features))

#     try:
#         cat_dict[converted_cat].extend(areas)
#     except:
#         cat_dict[converted_cat] = areas

#     # get area stats
#     areas = [area(feat["geometry"]) for feat in cat_features]
#     print("\nAREA STATS (in sq. metres)")
#     print('Min:', min(areas))
#     print('Max:', max(areas))
#     print('Mean:', np.mean(areas))
#     print('Std. Deviation:', np.std(areas))
#     for q in np.arange(10, 100, 20):
#         print(f"{q}th percentile:", np.percentile(areas, q))
#     plt.hist(areas, bins=20)
#     plt.ylabel("Frequency")
#     plt.xlabel("Size of Polygon")
#     plt.title(f'Histogram for {converted_cat}')
#     plt.show()
#     print("---")

### 3. Combine all Category GeoJSONs Together

This step combines all geojsons of different categories to create one single geojson. Needs the above code block to run before this is run.

Not a completely independent snippet/section. This should only be run on the outputs of the previous snippet.

In [ ]:
input_folder = f"/Users/Ishannangia/Desktop/three_dis_labelme_files"

label_dict = {
    "Distorted Image": "Distorted Image",
    "Bare Land": 'Bare Land',
    "Canopy Gap": 'Canopy Gap',
    "Plantation": 'Plantation',
}


main_dict = {'type': 'FeatureCollection',
             'features':[]}
all_geojsons = []

# collect all geojsons w.r.t. the categories/degradation indicators that have been defined
for k, v in label_dict.items():
    all_geojsons.append(os.path.join(input_folder, 
                                     f'{v}.geojson'))
    
# create one geojson using all geojsons
for geojson in all_geojsons:
    try:
        with open(geojson, "r") as file:
            f = json.load(file)
        main_dict['features'].extend(f["features"])
    except FileNotFoundError:
        pass
    
with open(os.path.join(input_folder, "final.geojson"), 'w') as f:
    dump(main_dict, f)

In [ ]:
# with open(os.path.join(input_folder, "final.geojson"), "r") as file:
#     f = json.load(file)

### Converting Vectors to Rasters

Converts vector files to raster files

Completely independenet section. Can be run without any dependence on other snippets.

In [ ]:
def get_meta_data_for_transform(extent_file_path, res):
    extent_file = gpd.read_file(extent_file_path)
    bbox = extent_file.total_bounds
    xmin, ymin, xmax, ymax = bbox

    w = (xmax - xmin) // res
    h = (ymax - ymin) // res

    out_meta = {
        "driver": "GTiff",
        "dtype": "uint8",
        "height": h,
        "width": w,
        "count": 1,
        "crs": extent_file.crs,
        "transform": transform.from_bounds(xmin, ymin, xmax, ymax, w, h),
        "compress": 'lzw'
    }
    return out_meta


def map_name_to_class(x):
    if 'Vegetated' in x:
        return 1
    elif 'Shaded' in x:
        return 2
    elif 'Bare' in x:
        return 3
    
    
def get_resolution_from_resolution_file_path(res_file_path):
    ds = rio.open(res_file_path)
    pixel_size_x, pixel_size_y = ds.res
    return pixel_size_x, pixel_size_y

In [ ]:
vector_names = ["/Users/ishannangia/Desktop/TfW/tifs and labels/burntSite_tifs/Canopy Gap: Bare Land.geojson",
                "/Users/ishannangia/Desktop/TfW/tifs and labels/burntSite_tifs/Canopy Gap: Shaded.geojson",
                "/Users/ishannangia/Desktop/TfW/tifs and labels/burntSite_tifs/Canopy Gap: Vegetated.geojson"]

In [ ]:
gdf_list = []
for vector_name in vector_names:
    gdf = gpd.read_file(vector_name)
    gdf_list.append(gdf)
    
main_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
main_gdf['classes'] = main_gdf.name.apply(map_name_to_class)

In [ ]:
extent_file_path = "../../Desktop/TfW/MhadeiAoi/MhadeiAoi.shp"
output_raster_path = "del_this_raster.tif"
res_file_path = "/Users/ishannangia/Desktop/TfW/tifs and labels/restorationSite.tif"

res = 0.00001 # desired resolution

In [ ]:
pixel_size_x, pixel_size_y = get_resolution_from_resolution_file_path(res_file_path)
res = pixel_size_x

In [ ]:
out_meta = get_meta_data_for_transform(extent_file_path, res)

with rio.open(output_raster_path, 'w+', **out_meta) as out:
    out_arr = out.read(1)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom, value) for geom, value in zip(main_gdf.geometry, main_gdf.classes))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)

In [ ]:
len(out_meta.keys())

In [ ]:
dataset = rio.open(output_raster_path)
dataset = rio.open('/Users/ishannangia/github_repos/Mhadei_Restoration/results/vec2raster.tif')

In [ ]:
show(dataset)